# ANALYSE DE TRANSFERMARKT

<b> Analyse des données transfermarkt issues de https://www.kaggle.com/davidcariboo/player-scores?select=competitions.csv </b>

In [1]:
import os
import zipfile
import swat 
import pandas as pd
from getpass import getpass
pd.set_option('display.max_colwidth',100000)
instance='viyawaves.sas.com'
cas_host ='https://'+instance+'/cas-shared-default-http/'
cas_port= 80

### CONNECTION TO CAS

In [2]:
user=input("Nom d'utilisateur:")
mdp=getpass("Mot de passe:")
s = swat.CAS(cas_host, cas_port, user, mdp)
s.sessionprop.setsessopt(caslib='CASUSER',timeout=31536000)

Nom d'utilisateur: frajoz
Mot de passe: ············


NOTE: 'CASUSER(frajoz)' is now the active caslib.


+ Elapsed: 0.000535s, user: 0.000336s, sys: 0.000163s, mem: 0.274mb

In [11]:
s.table.dropTable(name="TRANSFERMARKT_DATA",caslib="Public",quiet=True)
s.table.dropTable(name="JOIN_GAME_PLAYER",caslib="CASUSER",quiet=True)

+ Elapsed: 0.167s, user: 0.000416s, sys: 0.000129s, mem: 0.296mb

In [4]:
path_to_zip_file=os.fspath('C:\\Users\\frajoz\OneDrive - SAS\Domain\Sport Analytics\TRANSFERMARKT\ARCHIVE\\archive (21).zip')
directory_to_extract_to=os.fspath('C:\\Users\\frajoz\OneDrive - SAS\Domain\Sport Analytics\TRANSFERMARKT\DONNEES\\')

In [5]:
for file in os.listdir(directory_to_extract_to):
    sasname=file.replace('.csv','')
    s.read_csv(directory_to_extract_to+file,casout={'name':sasname,'replace':True})
    print(sasname)
    print(s.table.fetch(sasname,to=2))

NOTE: Cloud Analytic Services made the uploaded file available as table APPEARANCES in caslib CASUSER(frajoz).
NOTE: The table APPEARANCES has been created in caslib CASUSER(frajoz) from binary data uploaded to Cloud Analytic Services.
appearances
[Fetch]

 Selected Rows from Table APPEARANCES
 
    player_id    game_id  appearance_id competition_id  player_club_id  goals  \
 0    71420.0  2609372.0  2609372_71420            PO1          1301.0    0.0   
 1    71420.0  2609389.0  2609389_71420            PO1          1301.0    0.0   
 
    assists  minutes_played  yellow_cards  red_cards  
 0      0.0            70.0           1.0        0.0  
 1      0.0            90.0           0.0        0.0  

+ Elapsed: 0.00208s, user: 0.00101s, sys: 0.00102s, mem: 0.982mb
NOTE: Cloud Analytic Services made the uploaded file available as table CLUBS in caslib CASUSER(frajoz).
NOTE: The table CLUBS has been created in caslib CASUSER(frajoz) from binary data uploaded to Cloud Analytic Services.
clu

In [6]:
s.datastep.runcode("data JOIN_PLAYER_APP;merge APPEARANCES(in=in1 rename=(player_club_id=current_club_id))  PLAYERS;by player_id current_club_id;if in1;run;")

[InputCasTables]

             casLib         Name    Rows  Columns  \
 0  CASUSER(frajoz)  APPEARANCES  980509       10   
 1  CASUSER(frajoz)      PLAYERS   22431       14   
 
                                             casTable  
 0  CASTable('APPEARANCES', caslib='CASUSER(frajoz)')  
 1      CASTable('PLAYERS', caslib='CASUSER(frajoz)')  

[OutputCasTables]

             casLib             Name    Rows  Columns  Append Promoted  \
 0  CASUSER(frajoz)  JOIN_PLAYER_APP  980509       22     NaN        N   
 
                                                 casTable  
 0  CASTable('JOIN_PLAYER_APP', caslib='CASUSER(frajoz)')  

+ Elapsed: 1.1s, user: 3.65s, sys: 1.19s, mem: 143mb

In [12]:
s.datastep.runcode("data JOIN_GAME_PLAYER;merge JOIN_PLAYER_APP(in=in1)  GAMES;by game_id;if in1;run;")

NOTE: MERGE statement has more than one data set with repeats of BY values.
NOTE: Duplicate messages output by DATA step:
NOTE: MERGE statement has more than one data set with repeats of BY values.  (occurred 4 times)


[InputCasTables]

             casLib             Name    Rows  Columns  \
 0  CASUSER(frajoz)  JOIN_PLAYER_APP  980509       22   
 1  CASUSER(frajoz)            GAMES   42425       15   
 
                                                 casTable  
 0  CASTable('JOIN_PLAYER_APP', caslib='CASUSER(frajoz)')  
 1            CASTable('GAMES', caslib='CASUSER(frajoz)')  

[OutputCasTables]

             casLib              Name    Rows  Columns  Append Promoted  \
 0  CASUSER(frajoz)  JOIN_GAME_PLAYER  980510       35     NaN        N   
 
                                                  casTable  
 0  CASTable('JOIN_GAME_PLAYER', caslib='CASUSER(frajoz)')  

+ Elapsed: 2.07s, user: 7.51s, sys: 1.85s, mem: 113mb

In [13]:
s.table.promote(table='JOIN_GAME_PLAYER',caslib="CASUSER",Targetlib="CASUSER")

NOTE: Cloud Analytic Services promoted table JOIN_GAME_PLAYER in caslib CASUSER(frajoz) to table JOIN_GAME_PLAYER in caslib CASUSER(frajoz).


+ Elapsed: 0.12s, user: 0.0112s, sys: 0.108s, mem: 0.308mb

In [14]:
a=s.table.columnInfo('CLUBS')
df=a['ColumnInfo']

for k in ['CURRENT','HOME','AWAY']:
    s.datastep.runcode('data '+k+'_CLUB;set CLUBS;run;')
    for i in range(0,len(df)):
        var=df[(df.ID==i+1)]['Column'].iloc[0]
        newvar=k+"_"+var
        s.datastep.runcode("data "+k+"_CLUB;set "+k+"_CLUB;rename "+var+"="+newvar+";run;")

In [15]:
s.datastep.runcode('data JOIN_GAME_PLAYER1;merge JOIN_GAME_PLAYER(in=in1) CURRENT_CLUB; by current_club_id;if in1;run;')

s.datastep.runcode('data JOIN_CLUBS_HOME;merge JOIN_GAME_PLAYER1(in=in1) HOME_CLUB; by home_club_id;if in1;run;')
s.datastep.runcode('data JOIN_CLUBS_FULL;merge JOIN_CLUBS_HOME(in=in1) AWAY_CLUB; by away_club_id;if in1;run;')

[InputCasTables]

             casLib             Name    Rows  Columns  \
 0  CASUSER(frajoz)  JOIN_CLUBS_HOME  980510       63   
 1  CASUSER(frajoz)        AWAY_CLUB     379       15   
 
                                                 casTable  
 0  CASTable('JOIN_CLUBS_HOME', caslib='CASUSER(frajoz)')  
 1        CASTable('AWAY_CLUB', caslib='CASUSER(frajoz)')  

[OutputCasTables]

             casLib             Name    Rows  Columns  Append Promoted  \
 0  CASUSER(frajoz)  JOIN_CLUBS_FULL  980510       77     NaN        N   
 
                                                 casTable  
 0  CASTable('JOIN_CLUBS_FULL', caslib='CASUSER(frajoz)')  

+ Elapsed: 4.04s, user: 12.1s, sys: 3s, mem: 101mb

In [23]:
s.datastep.runcode('data PUBLIC.TRANSFERMARKT_DATA;merge JOIN_CLUBS_FULL(in=in1) COMPETITIONS(rename=(name=competition_name country_name=comp_country_name)); by competition_id;if in1;run;')

[InputCasTables]

             casLib             Name    Rows  Columns  \
 0  CASUSER(frajoz)  JOIN_CLUBS_FULL  980510       77   
 1  CASUSER(frajoz)     COMPETITIONS      42        8   
 
                                                 casTable  
 0  CASTable('JOIN_CLUBS_FULL', caslib='CASUSER(frajoz)')  
 1     CASTable('COMPETITIONS', caslib='CASUSER(frajoz)')  

[OutputCasTables]

    casLib                Name    Rows  Columns  Append Promoted  \
 0  Public  TRANSFERMARKT_DATA  980510       83     NaN        N   
 
                                           casTable  
 0  CASTable('TRANSFERMARKT_DATA', caslib='Public')  

+ Elapsed: 30.2s, user: 145s, sys: 17s, mem: 98.6mb

In [18]:
s.table.promote(table='TRANSFERMARKT_DATA',caslib="PUBLIC",targetLib="Public")


NOTE: Cloud Analytic Services promoted table TRANSFERMARKT_DATA in caslib CASUSER(frajoz) to table TRANSFERMARKT_DATA in caslib Public.


+ Elapsed: 0.211s, user: 0.00459s, sys: 0.205s, mem: 0.321mb

In [26]:
s.table.save(table={"name":"TRANSFERMARKT_DATA","caslib":"PUBLIC"},caslib="PUBLIC",name="TRANSFERMARKT_DATA.sashdat")

NOTE: Cloud Analytic Services saved the file TRANSFERMARKT_DATA.sashdat in caslib Public.


[caslib]

 'Public'

[name]

 'TRANSFERMARKT_DATA.sashdat'

+ Elapsed: 5.2s, user: 0.837s, sys: 2.24s, mem: 16.6mb

In [27]:
s.table.save(table={"name":"JOIN_GAME_PLAYER","caslib":"CASUSER"},caslib="CASUSER",name="JOIN_GAME_PLAYER.sashdat")

NOTE: Cloud Analytic Services saved the file JOIN_GAME_PLAYER.sashdat in caslib CASUSER(frajoz).


[caslib]

 'CASUSER(frajoz)'

[name]

 'JOIN_GAME_PLAYER.sashdat'

+ Elapsed: 1.33s, user: 0.287s, sys: 0.383s, mem: 16.6mb